In [ ]:
# Este bloco baixa os Arquivos CSV da copia online no github
# 
# E de ELT
import pandas as pd
import io
import requests

urls = {
    "2023": "https://raw.githubusercontent.com/scan4364/zologico2025.2/main/Projeto%20Integraliza%C3%A7%C3%A3o/elt/cadastro_dezembro_2023_rev02.csv",
    "2024": "https://raw.githubusercontent.com/scan4364/zologico2025.2/main/Projeto%20Integraliza%C3%A7%C3%A3o/elt/cadastro-de-ip-recife-31.12.2024.csv",
    "2025": "https://raw.githubusercontent.com/scan4364/zologico2025.2/main/Projeto%20Integraliza%C3%A7%C3%A3o/elt/cadastro-de-ip-recife-30.04.2025.csv"
}

configs = {
    "2023": {"sep": ",", "decimal": "."},
    "2024": {"sep": ";", "decimal": ","},
    "2025": {"sep": ";", "decimal": ","}
}

dfs = {}

for ano, url in urls.items():
    print(f"Baixando e processando {ano}...")
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        df_temp = pd.read_csv(
            io.StringIO(response.text),
            sep=configs[ano]["sep"],
            decimal=configs[ano]["decimal"],
            on_bad_lines='skip',
            low_memory=False  
        )
        
        df_temp['ano_referencia'] = ano
        df_temp.columns = [c.lower().strip().replace(' ', '_') for c in df_temp.columns]
        
        dfs[ano] = df_temp
        print(f"--> Sucesso! {len(df_temp)} linhas carregadas do ano {ano}.")
        
    except Exception as e:
        print(f"Erro ao processar {ano}: {e}")

Baixando e processando 2023...
--> Sucesso! 107010 linhas carregadas do ano 2023.
Baixando e processando 2024...
--> Sucesso! 110505 linhas carregadas do ano 2024.
Baixando e processando 2025...
--> Sucesso! 111666 linhas carregadas do ano 2025.


In [ ]:
# Este Bloco faz uma ligação no banco de dados

import urllib.parse
from sqlalchemy import create_engine

# Configurando a conexão
usuario_db = 'postgres.wbamssbwhweihxxthfnm'                  # Usuário padrão
senha_db = 'bd2025.1'                # Senha
host_db = 'aws-0-us-west-2.pooler.supabase.com'                     # Padrão
porta_db = '5432'                    # Padrão
nome_banco_db = 'postgres'           # O banco criado


# Monta a string de conexão segura
DB_CONNECTION_STR = f'postgresql://{usuario_db}:{senha_db}@{host_db}:{porta_db}/{nome_banco_db}?client_encoding=utf8'

# Cria a engine e testa
engine = create_engine(DB_CONNECTION_STR)

# Teste de conexão
try:
    with engine.connect() as connection:
        print("Conexão com o Supabase Postegre realizada com sucesso! 🚀")
except Exception as e:
    print(f"Erro ao conectar: {e}")

Conexão com o PostgreSQL realizada com sucesso! 🚀


In [ ]:

# Aqui carregamos as tabelas extraidas em uma só tabela, do tipo estrategica no banco de dados
# 
# L de ELT

for ano, df in dfs.items():
    tabela = f"stg_iluminacao_{ano}"
    print(f"⏳ Iniciando carga de {ano} para a tabela '{tabela}'...")
    
    try:
        # Aqui criamos a tabela no banco automaticamente
        df.to_sql(
            name=tabela,
            con=engine,
            if_exists='replace',  # Se a tabela já existi, apaga e cria de novo
            index=False,          # Não cria uma coluna extra para o índice do Pandas
            chunksize=1000        # Envia de 1000 em 1000 linhas para não travar a memória
        )
        print(f"✅ Sucesso! Tabela '{tabela}' carregada no Supabase PostgreSQL.")
        
    except Exception as e:
        print(f"❌ Erro ao carregar {ano}: {e}")

⏳ Iniciando carga de 2023 para a tabela 'stg_iluminacao_2023'...
✅ Sucesso! Tabela 'stg_iluminacao_2023' carregada no PostgreSQL.
⏳ Iniciando carga de 2024 para a tabela 'stg_iluminacao_2024'...
✅ Sucesso! Tabela 'stg_iluminacao_2024' carregada no PostgreSQL.
⏳ Iniciando carga de 2025 para a tabela 'stg_iluminacao_2025'...
✅ Sucesso! Tabela 'stg_iluminacao_2025' carregada no PostgreSQL.
